# Asteroid Light Curves Examples - Part 2

This notebook uses the same asteroid light curve data as part 1 but looks at several unsupervised techniques.

## Objectives

- Understand several techniques that might be available when you have little or no labeled data.